# Algoritmo de Búsqueda A-estrella / A-star

Es un algoritmo para buscar en grafos la ruta más corta desde un punto/nodo de origen a un punto/nodo final.
Para entender este algoritmo, se debe conocer primero algoritmos como el **Algoritmo de Dijkstra** y la **Búsqueda del Primero Mejor (Best-First-Search)**  
## Búsqueda de Anchura Primero (Breadth-First-Search)
Explora igualmente en todas las direcciones. Este es un algoritmo increíblemente útil, no solo para la búsqueda de rutas regulares. La idea principal es mantener un registro de un anillo expandido llamado **frontier**. En una cuadrícula este proceso es denominado **flood fill**. Este se **expande** repetitivamente a través de los nodos vecinos hasta encontrar el nodo objetivo.  

Un ejemplo de código sencillo:  
```python
frontier = Queue()
frontier.put(start)
visited = {}
visited[start] = True

while not frontier.empty():
   current = frontier.get()
   for next in graph.neighbors(current):
      if next not in visited:
         frontier.put(next)
         visited[next] = True
```
Y una animación al respecto de lo que este algoritmo haría sería:  
![Imgur](https://i.imgur.com/tTUhJdQ.gif)

En esencia este tipo de búsqueda también se implementa en A-estrella pero, cómo se encuentra la **ruta más corta**?  

Esto se puede hacer guardando un registro de cuál nodo fue el nodo anterior para llegar al actual. En este caso mediante el diccionario ```came_from```: 
```python
frontier = Queue()
frontier.put(start )
came_from = {}
came_from[start] = None

while not frontier.empty():
   current = frontier.get()
   for next in graph.neighbors(current):
      if next not in came_from:
         frontier.put(next)
         came_from[next] = current

# path construction

current = goal 
path = []
while current != start: 
   path.append(current)
   current = came_from[current]
path.append(start) # optional
```
![Imgur](https://i.imgur.com/xJwtaO4.png)

### Early Exit
Consiste en detener el algoritmo de búsqueda una vez se encuentra el objetivo, de esta manera:  

![Imgur](https://i.imgur.com/16adian.png)

La implementación de Early Exit en un algoritmo de búsqueda puede ser tan sencillo como: 
```python
frontier = Queue()
frontier.put(start)
came_from = {}
came_from[start] = None

while not frontier.empty():
   current = frontier.get()

   # Early Exit
   if current == goal: 
      break           

   for next in graph.neighbors(current):
      if next not in came_from:
         frontier.put(next)
         came_from[next] = current
```

### Costos de Movimiento
Esto implementa un nuevo factor por considerar en los algoritmos de búsqueda, pues la ruta más trivial no siempre puede resultar en la más corta si se toman en cuenta los costos que esta implica. En el siguiente ejemplo, se contrastan dos maneras de llegar desde un punto de inicio a un punto objetivo; a la izquierda se toma en cuenta la mejor ruta de acuerdo al número de pasos requeridos, y a la derecha se considera la distancia o costo de la ruta, tomando como referencia que un cuadro **beis implica 1 movimiento** y uno **verde implica 5 movimientos**:  

![Imgur](https://i.imgur.com/uY1xcb8.png)


## Algoritmo de Disjktra (Búsqueda de Costo Uniforme)
Es una implementación del algoritmo de Búsqueda de Anchura Primero que toma en cuenta los costos que implica tomar una ruta, y selecciona la que tenga un menor costo. Este algoritmo **siempre** se asegura de encontrar una ruta en el espacio que cumpla con ser la mejor, en caso de que exista una o más rutas posibles. Un ejemplo de una implementación en código:  
```python
frontier = PriorityQueue()
frontier.put(start, 0)
came_from = {}
cost_so_far = {}
came_from[start] = None
cost_so_far[start] = 0

while not frontier.empty():
   current = frontier.get()

   if current == goal:
      break
   
   for next in graph.neighbors(current):
      new_cost = cost_so_far[current] + graph.cost(current, next)
      if next not in cost_so_far or new_cost < cost_so_far[next]:
         cost_so_far[next] = new_cost
         priority = new_cost
         frontier.put(next, priority)
         came_from[next] = current
```  

Al cambiar la cola normal por una cola de prioridad, esto cambia el comportamiento de la expansión de la frontera.

![Imgur](https://i.imgur.com/DR6Rihf.gif)


## Búsqueda Heurística
Con Breadth First Search y el algoritmo de Dijkstra, la frontera se expande en todas las direcciones. Esta es una opción razonable si está tratando de encontrar una ruta a todas las ubicaciones o a muchas ubicaciones. Sin embargo, un caso común es encontrar una ruta a una sola ubicación. Ahora se necesita hacer que la frontera se expanda hacia la meta más de lo que se expande en otras direcciones. Primero, se define una función heurística que indica **qué tan cerca se encuentra un punto inicial al objetivo**:  
```python
def heuristic(a, b):
   # Manhattan distance on a square grid
   return abs(a.x - b.x) + abs(a.y - b.y)
```

En el algoritmo de Dijkstra, se utiliza la distancia actual desde el punto inicial para el ordenamiento de la cola de prioridad. Aquí, en cambio, en Greedy Best First Search, se utiliza la distancia estimada hasta el objetivo para el orden de la cola de prioridad. Primero se explorará la ubicación **más cercana al objetivo**. El código utiliza la cola de prioridad del algoritmo de Dijkstra pero sin ```cost_so_far```:  
```python
frontier = PriorityQueue()
frontier.put(start, 0)
came_from = {}
came_from[start] = None

while not frontier.empty():
   current = frontier.get()

   if current == goal:
      break
   
   for next in graph.neighbors(current):
      if next not in came_from:
         # Priority is now set by an heuristic
         priority = heuristic(goal, next)
         frontier.put(next, priority)
         came_from[next] = current
```
Una animación del funcionamiento de este código sería:  

![Imgur](https://i.imgur.com/Yd8RVqs.gif)

Y con obstáculos:  

![Imgur](https://i.imgur.com/YWlrcim.gif)
Este algoritmo no asegura que la ruta encontrada sea la más corta, en comparación con el Algoritmo de Dijkstra.

## Algoritmo A*
El Algoritmo de Dijkstra trabaja bien encontrando la ruta más corta, **pero gasta tiempo buscando en otras direcciones que resultan no ser útiles para el resultado**. Greedy Best First en direcciones más favorables **pero no asegura la ruta más corta**. El algoritmo A* utiliza ambas características: la distancia actual desde el inicio y distancia estimada del objetivo. La fórmula se puede definir el algoritmo como ```f(n) = g(n) + h(n)```. El código varía un poco al del Algoritmo de Dijkstra:  

```python
frontier = PriorityQueue()
frontier.put(start, 0)
came_from = {}
cost_so_far = {}
came_from[start] = None
cost_so_far[start] = 0

while not frontier.empty():
   current = frontier.get()

   if current == goal:
      break
   
   for next in graph.neighbors(current):
      new_cost = cost_so_far[current] + graph.cost(current, next)
      if next not in cost_so_far or new_cost < cost_so_far[next]:
         cost_so_far[next] = new_cost
         # now priority includes an heuristic estimation besides Dijkstra implementation
         priority = new_cost + heuristic(goal, next)
         frontier.put(next, priority)
         came_from[next] = current
```

Una comparación de A* con el Algoritmo de Dijkstra y el Greedy Best First Search sería:


![Imgur](https://i.imgur.com/TyTCrkf.png)




# Referencia

Toda la teoría consultada, ejemplos de código y animaciones fueron obtenidas del siguiente sitio [Introduction to A* From Amit’s Thoughts on Pathfinding](http://theory.stanford.edu/~amitp/GameProgramming/AStarComparison.html).  
Además, este otro sitio [Introduction to A* from Red Blob Games](https://www.redblobgames.com/pathfinding/a-star/introduction.html) 